In [1]:
from ae_detector import *
import warnings
warnings.filterwarnings('ignore')

/common/software/install/migrated/anaconda/python3-2020.07-mamba/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


## Anomaly detection with CoDSA

In [2]:
# Load and preprocess datasets
# MNIST dataset

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

mnist_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)

# USPS dataset
usps_data = datasets.USPS(root='data', train=True, download=True, transform=transform)

In [4]:
def extract_digits(dataset, digit):
    imgs = []
    for img, label in dataset:
        if label == digit:
            imgs.append(img.numpy())
    return np.array(imgs)

# Extract digit '3' (normal) and '5' (anomaly)
mnist_3 = extract_digits(mnist_data, 3)
mnist_5 = extract_digits(mnist_data, 5)
usps_3 = extract_digits(usps_data, 3)
usps_5 = extract_digits(usps_data, 5)

In [ ]:
#!python train_sample.py #train and sample images if the synthetic data are not ready

In [19]:
# Path to your generated images directory
image_dir = 'generated_images/mnist'

# Get sorted list of image filenames
image_files = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.endswith('.png')
])

# Read images into list and convert to NumPy array
images_list = []
for img_file in image_files:
    with Image.open(img_file).convert('L') as img:  # 'L' mode for grayscale
        img_array = np.array(img)
        images_list.append(img_array)

# Convert list to single NumPy array (N, Height, Width)
mnist_g = np.stack(images_list, axis=0)

print('Array shape:', mnist_g.shape)

# Path to your generated images directory
image_dir = 'generated_images/usps'

# Get sorted list of image filenames
image_files = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.endswith('.png')
])

# Read images into list and convert to NumPy array
images_list = []
for img_file in image_files:
    with Image.open(img_file).convert('L') as img:  # 'L' mode for grayscale
        img_array = np.array(img)
        images_list.append(img_array)

# Convert list to single NumPy array (N, Height, Width)
usps_g = np.stack(images_list, axis=0)

print('Array shape:', usps_g.shape)

Array shape: (10000, 28, 28)
Array shape: (15000, 28, 28)


In [92]:
import numpy as np
import torch

set_seed(42)
res=[]
# Set the random seed for reproducibility
for j in range(10):
    np.random.seed(j)
    all_results=[]
    # --- Randomly select 150 "normal" samples from class-3 datasets ---
    idx_mnist3 = np.random.choice(len(mnist_3), 150, replace=False)
    idx_usps3   = np.random.choice(len(usps_3),   150, replace=False)

    # Compute the remaining indices for training
    train_idx_mnist3 = np.setdiff1d(np.arange(len(mnist_3)), idx_mnist3)
    train_idx_usps3   = np.setdiff1d(np.arange(len(usps_3)),   idx_usps3)

    # Build the normal training set
    X_train_normal = np.concatenate((
        mnist_3[train_idx_mnist3],
        usps_3[  train_idx_usps3]
    ), axis=0)
    s_train = np.array(
        [1] * len(train_idx_mnist3) +
        [0] * len(train_idx_usps3)
    )

    # --- Build the "normal" portion of the test set (150 each) ---
    X_normal = np.concatenate((
        mnist_3[idx_mnist3],
        usps_3[  idx_usps3]
    ), axis=0)

    # --- Randomly select 150 "anomaly" samples from class-5 datasets ---
    idx_mnist5 = np.random.choice(len(mnist_5), 150, replace=False)
    idx_usps5   = np.random.choice(len(usps_5),   150, replace=False)

    X_anomaly = np.concatenate((
        mnist_5[idx_mnist5],
        usps_5[  idx_usps5]
    ), axis=0)

    # --- Combine to form the full test set ---
    X_test  = np.concatenate((X_normal, X_anomaly), axis=0)
    s_test  = np.array(
        [1] * len(idx_mnist3) + [0] * len(idx_usps3) +
        [1] * len(idx_mnist5) + [0] * len(idx_usps5)
    )

    # Labels: 0 = normal, 1 = anomaly
    y_normal  = np.zeros(len(X_normal))
    y_anomaly = np.ones(len(X_anomaly))
    y_test    = np.concatenate((y_normal, y_anomaly), axis=0)
    
    nr=X_train_normal.shape[0]
    s_tr=s_train#[y_train==0]
    pk=np.array([sum(s_tr==1)/nr,sum(s_tr==0)/nr])
    q_k=np.array([0.5,0.5])


    detector = AutoencoderAnomalyDetector(seed=0)
    detector.fit(X_train_normal)
    y_pred = detector.predict(X_test)
    overall_metrics = detector.evaluate(X_test, y_test)
    groups = ['USPS', 'MNIST']
    fairness_metrics = {}
    for group_value, group_name in zip([0, 1], groups):
        idx = (s_test == group_value)
        fairness_metrics[group_name] =detector.evaluate(X_test[idx], y_test[idx])
    results = {
        'Metric': ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN'],
        'Overall': [overall_metrics[m] for m in ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN']],
        'USPS': [fairness_metrics['USPS'][m] for m in ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN']],
        'MNIST': [fairness_metrics['MNIST'][m] for m in ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN']],
    }

    df_origin = pd.DataFrame(results)

    all_results.append(df_origin)
    
    for sample_size in [6000,12000]:
        ratio = nr*(q_k-pk)/(sample_size)+ q_k
        X_all= torch.tensor(np.vstack([X_train_normal,
                                       mnist_g[:int(sample_size*ratio[0]), np.newaxis, :, :]/255,
                                       usps_g[:int(sample_size*ratio[1]), np.newaxis, :, :]/255])).float()

        detector = AutoencoderAnomalyDetector(seed=0)
        detector.fit(X_all)
        y_pred = detector.predict(X_test)
        overall_metrics = detector.evaluate(X_test, y_test)
        groups = ['USPS', 'MNIST']
        fairness_metrics = {}
        for group_value, group_name in zip([0, 1], groups):
            idx = (s_test == group_value)
            fairness_metrics[group_name] =detector.evaluate(X_test[idx], y_test[idx])
        results = {
            'Metric': ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN'],
            'Overall': [overall_metrics[m] for m in ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN']],
            'USPS': [fairness_metrics['USPS'][m] for m in ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN']],
            'MNIST': [fairness_metrics['MNIST'][m] for m in ['AUC', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN', 'FN']],
        }
        
        df_metrics = pd.DataFrame(results)

        all_results.append(df_metrics)
        
    res.append(all_results)
    

Epoch 1/20  Loss: 0.219872
Epoch 2/20  Loss: 0.210679
Epoch 3/20  Loss: 0.182353
Epoch 4/20  Loss: 0.129671
Epoch 5/20  Loss: 0.086223
Epoch 6/20  Loss: 0.069565
Epoch 7/20  Loss: 0.064560
Epoch 8/20  Loss: 0.062426
Epoch 9/20  Loss: 0.061118
Epoch 10/20  Loss: 0.060215
Epoch 11/20  Loss: 0.059359
Epoch 12/20  Loss: 0.058344
Epoch 13/20  Loss: 0.057148
Epoch 14/20  Loss: 0.055836
Epoch 15/20  Loss: 0.054536
Epoch 16/20  Loss: 0.053227
Epoch 17/20  Loss: 0.051900
Epoch 18/20  Loss: 0.050632
Epoch 19/20  Loss: 0.049478
Epoch 20/20  Loss: 0.048507


/users/1/tianx/DA/CoDSA/Real examples/Anomaly detection/ae_detector.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X_train, dtype=torch.float32, device=self.device)


Epoch 1/20  Loss: 0.165975
Epoch 2/20  Loss: 0.126275
Epoch 3/20  Loss: 0.077699
Epoch 4/20  Loss: 0.055132
Epoch 5/20  Loss: 0.047801
Epoch 6/20  Loss: 0.045770
Epoch 7/20  Loss: 0.044782
Epoch 8/20  Loss: 0.044074
Epoch 9/20  Loss: 0.043364
Epoch 10/20  Loss: 0.042415
Epoch 11/20  Loss: 0.041169
Epoch 12/20  Loss: 0.039692
Epoch 13/20  Loss: 0.038262
Epoch 14/20  Loss: 0.036907
Epoch 15/20  Loss: 0.035445
Epoch 16/20  Loss: 0.033882
Epoch 17/20  Loss: 0.032450
Epoch 18/20  Loss: 0.031303
Epoch 19/20  Loss: 0.030311
Epoch 20/20  Loss: 0.029395
Epoch 1/20  Loss: 0.152035


/users/1/tianx/DA/CoDSA/Real examples/Anomaly detection/ae_detector.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X_train, dtype=torch.float32, device=self.device)


Epoch 2/20  Loss: 0.085764
Epoch 3/20  Loss: 0.051171
Epoch 4/20  Loss: 0.043597
Epoch 5/20  Loss: 0.041985
Epoch 6/20  Loss: 0.041073
Epoch 7/20  Loss: 0.039897
Epoch 8/20  Loss: 0.038146
Epoch 9/20  Loss: 0.036049
Epoch 10/20  Loss: 0.033781
Epoch 11/20  Loss: 0.031618
Epoch 12/20  Loss: 0.030042
Epoch 13/20  Loss: 0.028685
Epoch 14/20  Loss: 0.027350
Epoch 15/20  Loss: 0.026048
Epoch 16/20  Loss: 0.024927
Epoch 17/20  Loss: 0.023979
Epoch 18/20  Loss: 0.023105
Epoch 19/20  Loss: 0.022249
Epoch 20/20  Loss: 0.021442
Epoch 1/20  Loss: 0.219898
Epoch 2/20  Loss: 0.210673
Epoch 3/20  Loss: 0.182158
Epoch 4/20  Loss: 0.129062
Epoch 5/20  Loss: 0.085638
Epoch 6/20  Loss: 0.069326
Epoch 7/20  Loss: 0.064521
Epoch 8/20  Loss: 0.062470
Epoch 9/20  Loss: 0.061219
Epoch 10/20  Loss: 0.060378
Epoch 11/20  Loss: 0.059535
Epoch 12/20  Loss: 0.058536
Epoch 13/20  Loss: 0.057363
Epoch 14/20  Loss: 0.056142
Epoch 15/20  Loss: 0.054944
Epoch 16/20  Loss: 0.053722
Epoch 17/20  Loss: 0.052383
Epoch 18/

...

Epoch 2/20  Loss: 0.085425
Epoch 3/20  Loss: 0.050709
Epoch 4/20  Loss: 0.043464
Epoch 5/20  Loss: 0.041947
Epoch 6/20  Loss: 0.041081
Epoch 7/20  Loss: 0.040002
Epoch 8/20  Loss: 0.038363
Epoch 9/20  Loss: 0.036289
Epoch 10/20  Loss: 0.034107
Epoch 11/20  Loss: 0.031843
Epoch 12/20  Loss: 0.030171
Epoch 13/20  Loss: 0.028828
Epoch 14/20  Loss: 0.027507
Epoch 15/20  Loss: 0.026251
Epoch 16/20  Loss: 0.025114
Epoch 17/20  Loss: 0.024133
Epoch 18/20  Loss: 0.023299
Epoch 19/20  Loss: 0.022566
Epoch 20/20  Loss: 0.021885
Epoch 1/20  Loss: 0.219883
Epoch 2/20  Loss: 0.210620
Epoch 3/20  Loss: 0.182171
Epoch 4/20  Loss: 0.129289
Epoch 5/20  Loss: 0.085733
Epoch 6/20  Loss: 0.069247
Epoch 7/20  Loss: 0.064338
Epoch 8/20  Loss: 0.062235
Epoch 9/20  Loss: 0.060972
Epoch 10/20  Loss: 0.060113
Epoch 11/20  Loss: 0.059253
Epoch 12/20  Loss: 0.058230
Epoch 13/20  Loss: 0.057017
Epoch 14/20  Loss: 0.055653
Epoch 15/20  Loss: 0.054274
Epoch 16/20  Loss: 0.052851
Epoch 17/20  Loss: 0.051400
Epoch 18/

/users/1/tianx/DA/CoDSA/Real examples/Anomaly detection/ae_detector.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X_train, dtype=torch.float32, device=self.device)


Epoch 2/20  Loss: 0.126273
Epoch 3/20  Loss: 0.077561
Epoch 4/20  Loss: 0.055054
Epoch 5/20  Loss: 0.047707
Epoch 6/20  Loss: 0.045666
Epoch 7/20  Loss: 0.044703
Epoch 8/20  Loss: 0.044030
Epoch 9/20  Loss: 0.043363
Epoch 10/20  Loss: 0.042460
Epoch 11/20  Loss: 0.041260
Epoch 12/20  Loss: 0.039830
Epoch 13/20  Loss: 0.038411
Epoch 14/20  Loss: 0.036997
Epoch 15/20  Loss: 0.035410
Epoch 16/20  Loss: 0.033691
Epoch 17/20  Loss: 0.032170
Epoch 18/20  Loss: 0.030936
Epoch 19/20  Loss: 0.029838
Epoch 20/20  Loss: 0.028793
Epoch 1/20  Loss: 0.152018
Epoch 2/20  Loss: 0.085709


/users/1/tianx/DA/CoDSA/Real examples/Anomaly detection/ae_detector.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X_train, dtype=torch.float32, device=self.device)


Epoch 3/20  Loss: 0.051022
Epoch 4/20  Loss: 0.043518
Epoch 5/20  Loss: 0.041963
Epoch 6/20  Loss: 0.040980
Epoch 7/20  Loss: 0.039669
Epoch 8/20  Loss: 0.037748
Epoch 9/20  Loss: 0.035565
Epoch 10/20  Loss: 0.033298
Epoch 11/20  Loss: 0.031427
Epoch 12/20  Loss: 0.030107
Epoch 13/20  Loss: 0.028882
Epoch 14/20  Loss: 0.027544
Epoch 15/20  Loss: 0.026143
Epoch 16/20  Loss: 0.024846
Epoch 17/20  Loss: 0.023756
Epoch 18/20  Loss: 0.022872
Epoch 19/20  Loss: 0.022115
Epoch 20/20  Loss: 0.021457


In [93]:
res_np=np.array([[res[x][y].iloc[0,1:].tolist() for y in range(3)] for x in range(10)])

### Summary statistics for Table 3

In [95]:
np.mean(res_np[:,:,:],0)

array([[0.82663667, 0.83212444, 0.85461778],
       [0.88604778, 0.89485778, 0.89493333],
       [0.90141444, 0.89567556, 0.93204   ]])

In [96]:
np.std(res_np[:,:,:],0)

array([[0.0179769 , 0.0208599 , 0.02859189],
       [0.01419213, 0.01588405, 0.02195932],
       [0.01117417, 0.01497703, 0.01761798]])